# 多次元配列としてのテンソル(3.2 ~ 3.4までの範囲)

In [1]:
import torch

## 3.2.1 Python listからPyTorchテンソルへ

In [2]:
a = [1.0, 2.0, 3.0]
a

[1.0, 2.0, 3.0]

## 3.2.2 テンソルの作成

In [6]:
a = torch.ones(3) # 1が3つ格納されたテンソルを生成
print(a)
print(a[1])
print(float(a[1]))
a[2] = 2.0
print(a)

tensor([1., 1., 1.])
tensor(1.)
1.0
tensor([1., 1., 2.])


## 3.3テンソルの一部指定や取り出し

In [ ]:
some_list = list(range(6))
# print(some_list)
print(some_list[:]) # 全要素
print(some_list[1:4]) # [1, 2, 3]
print(some_list[1:]) # [1, 2, 3, 4, 5]
print(some_list[:4]) # [0, 1, 2, 3]
print(some_list[:-1]) # [0, 1, 2, 3, 4]
print(some_list[1:4:2]) # [1, 3]


In [ ]:
points = torch.tensor([[4.0, 1.0], [5.0, 3.0], [2.0, 1.0]])
# print(points)
# print(points[1:])
# print(points[1:, :])
# print(points[1:, 0])
print(points.shape) # [3, 2]
points_none = points[None] # 新たな次元を追加
print(points_none.shape) # [1, 3, 2]

## 3.4 名前付きテンソル
どの次元にどのデータが格納されているのかを明らかにする

* tensorは後ろから3次元目にチャンネルが来る
* 画像のバッチ: 複数の画像を一度に処理するためにグループ化したもの



In [ ]:
img_t = torch.randn(3, 5, 5) # channel, rows, comumns
weights = torch.tensor([0.2126, 0.7152, 0.0722])
print(img_t)

In [ ]:
batch_t = torch.randn(2, 3, 5, 5)
print(batch_t)

**カラー画像をグレースケールに変換**  
チャンネルに沿って平均値を求める

In [ ]:
print(img_t.shape) # [3, 5, 5]
print(batch_t.shape) # [2, 3, 5, 5]

img_gray_naive = img_t.mean(-3) #
# print(img_gray_naive)
print(img_gray_naive.shape) # [5, 5]

batch_gray_navie = batch_t.mean(-3)
# print(batch_gray_navie)
print(batch_gray_navie.shape) # [2, 5, 5]

In [39]:
unsqueeze_weights = weights.unsqueeze(-1).unsqueeze(-1)

# print(unsqueeze_weights.shape) # 3, 1, 1
img_weights = (img_t * unsqueeze_weights) #  各チャンネルのすべての行列に対応するweightsの値をかける
# print(img_weights.shape) # 3, 5, 5
img_gray_weighted = img_weights.sum(-3) # チャンネルを基準に行列を合計
# print(img_gray_weighted) # 5, 5

batch_weights = (batch_t * unsqueeze_weights) #  各チャンネルのすべての行列に対応するweightsの値をかける
print(batch_weights.shape) # 2, 3, 5, 5
batch_gray_weighted = batch_weights.sum(-3)
print(batch_gray_weighted.shape) # 2, 5, 5

torch.Size([3, 5, 5])
torch.Size([2, 3, 5, 5])
torch.Size([2, 5, 5])


**torch.einsum**  
演算を示す矢印（->）の左側に入力テンソルの次元、右側に出力テンソルの次元を配置します。

In [9]:
img_gray_weighted_fancy = torch.einsum('...chw, c->...hw', img_t, weights)
batch_gray_weighted_fancy = torch.einsum('...bchw, c->...hw', batch_t, weights)
print(img_gray_weighted_fancy.shape)
print(batch_gray_weighted_fancy.shape)

torch.Size([5, 5])
torch.Size([5, 5])


### 次元に名前を付ける

In [ ]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channel'])
print(weights_named)

### テンソルに名前を追加

In [ ]:
img_named = img_t.refine_names(..., 'channel', 'height', 'weight')
batch_named = batch_t.refine_names('batch', 'channel', 'height', 'weight')
print(img_named)
print(img_named.shape)
print(batch_named)

In [15]:
print(img_named[..., :3].shape) # 3, 5, 3

torch.Size([3, 5, 3])


# メモ
**ブロードキャスト**  
サイズの小さい行列がサイズの大きいサイズに自動的に変換して計算する仕組み